In [1]:
# 
import pickle
import os
import numpy as np
import matplotlib.pyplot as plt


with open('ANDI_DATA/feature_v2_trainset.pkl', 'rb') as f:
    train_dataset = pickle.load(f)
    
with open('ANDI_DATA/feature_v2_valset.pkl', 'rb') as f:
    val_dataset = pickle.load(f)

with open('ANDI_DATA/feature_v2_testset.pkl', 'rb') as f:
    test_dataset = pickle.load(f)



In [2]:
import tensorflow as tf
from tensorflow import reduce_sum

def weighted_sum_data_k(
    data,
    coefficients: list = [1,1,1,1,1,1,1,1]
):

    for idx, coefficient in enumerate(coefficients):
        data[..., idx] = data[..., idx] * coefficient

    data = reduce_sum(
        data,
        axis=len(data.shape) - 1,
        keepdims=True
    )

    return data

In [3]:
train_input = weighted_sum_data_k(train_dataset['x'])
train_input = tf.reshape(train_input,(train_input.shape[0],1,191,1))
train_output = train_dataset['alpha']

val_input = weighted_sum_data_k(val_dataset['x'])
val_input = tf.reshape(val_input,(val_input.shape[0],1,191,1))
val_output = val_dataset['alpha']

test_input = weighted_sum_data_k(test_dataset['x'])
test_input = tf.reshape(test_input,(test_input.shape[0],1,191,1))
test_output = test_dataset['alpha']

In [4]:
print('train_input shape = {}'.format(train_input.shape))
print('train_output shape = {}'.format(train_output.shape))
print('val_input shape = {}'.format(val_input.shape))
print('val_output shape = {}'.format(val_output.shape))
print('test_input shape = {}'.format(test_input.shape))
print('test_output shape = {}'.format(test_output.shape))

train_input shape = (436000, 1, 191, 1)
train_output shape = (436000, 200)
val_input shape = (116000, 1, 191, 1)
val_output shape = (116000, 200)
test_input shape = (72000, 1, 191, 1)
test_output shape = (72000, 200)


In [8]:
import os
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Input, Conv1D, BatchNormalization, Activation, MaxPooling1D, Flatten, TimeDistributed
from tensorflow.keras.layers import Dropout, Bidirectional, LSTM, Attention, Multiply, Reshape, Permute, Dense, AdditiveAttention
from tensorflow.keras import regularizers
from tensorflow.keras.backend import clear_session
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

MODEL_SAVE_FOLDER_PATH = os.path.abspath('./') + '/models/'
if not os.path.exists(MODEL_SAVE_FOLDER_PATH):
    os.mkdir(MODEL_SAVE_FOLDER_PATH)

model_path = MODEL_SAVE_FOLDER_PATH + 'model_alpha.hdf5' 

cb_checkpoint = ModelCheckpoint(filepath=model_path, monitor='val_loss', mode = 'min',\
                                verbose=1, save_best_only=True)
cb_early_stopping = EarlyStopping(monitor='val_loss', mode = 'min', patience= 10)

# Compile model

rlr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience= 5, min_lr=1e-7, \
                        verbose=1, min_delta=1e-5)


l1 = 1e-5
l2 = 1e-4
lr = 1e-4
kernel_size=8
strides=1
activation='relu'




inputs = Input((1, 191, 1))
x = TimeDistributed(Conv1D(256, kernel_size=kernel_size, input_shape=(None, 191, 1)))(inputs)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = TimeDistributed(MaxPooling1D(2))(x)
x = TimeDistributed(Conv1D(512, kernel_size=kernel_size))(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = TimeDistributed(MaxPooling1D(2))(x)
x = TimeDistributed(Conv1D(1024, kernel_size=kernel_size))(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = TimeDistributed(MaxPooling1D(2))(x)
x = TimeDistributed(Flatten())(x)
x = Dropout(0.25)(x)
x = Bidirectional(LSTM(1024, activation='relu', return_sequences=True))(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)
x = Bidirectional(LSTM(1024, activation='relu', return_sequences=False))(x)
x = Reshape((2048, 1))(x)
x = Permute((2, 1))(x)
x = Reshape((-1, 2048))(x)

attention_result = AdditiveAttention()([x, x])
x = Multiply()([x, attention_result])

x = Permute((2, 1))(x)
x = Reshape((-1, 2048))(x)
x = Flatten()(x)

x = BatchNormalization()(x)
outputs = Dense(
        200,
        activation=activation,
        kernel_regularizer=regularizers.l1_l2(l1=l1, l2=l2)
    )(x)

model = Model(inputs=inputs, outputs=outputs)
model.summary()


model.compile(optimizer= Adam(lr),
                  loss='mae')

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 1, 191, 1)]  0                                            
__________________________________________________________________________________________________
time_distributed_14 (TimeDistri (None, 1, 184, 256)  2304        input_3[0][0]                    
__________________________________________________________________________________________________
batch_normalization_10 (BatchNo (None, 1, 184, 256)  1024        time_distributed_14[0][0]        
__________________________________________________________________________________________________
activation_6 (Activation)       (None, 1, 184, 256)  0           batch_normalization_10[0][0]     
____________________________________________________________________________________________

In [9]:
history = model.fit(train_input, train_output, epochs = 1000, batch_size= 1024, verbose=1, \
                   validation_data=(val_input, val_output),callbacks=[cb_checkpoint, cb_early_stopping, rlr])

Epoch 1/1000
426/426 [==============================] - 61s 135ms/step - loss: 0.3597 - val_loss: 0.3163

Epoch 00001: val_loss improved from inf to 0.31635, saving model to /workspace/Andi challenge2/본선/andi challenge 2024/models/model_alpha.hdf5
Epoch 2/1000
426/426 [==============================] - 57s 135ms/step - loss: 0.2427 - val_loss: 0.2407

Epoch 00002: val_loss improved from 0.31635 to 0.24074, saving model to /workspace/Andi challenge2/본선/andi challenge 2024/models/model_alpha.hdf5
Epoch 3/1000
426/426 [==============================] - 57s 135ms/step - loss: 0.1923 - val_loss: 0.1913

Epoch 00003: val_loss improved from 0.24074 to 0.19135, saving model to /workspace/Andi challenge2/본선/andi challenge 2024/models/model_alpha.hdf5
Epoch 4/1000
426/426 [==============================] - 57s 135ms/step - loss: 0.1653 - val_loss: 0.1618

Epoch 00004: val_loss improved from 0.19135 to 0.16179, saving model to /workspace/Andi challenge2/본선/andi challenge 2024/models/model_alpha.h

KeyboardInterrupt: 